Install XGBoost

In [ ]:
!pip install xgboost

In [1]:
import pandas as pd

Read the training file

In [2]:
train = pd.read_csv('cybersecurity_training.csv',sep='|')
train

,alert_ids,client_code,notified,categoryname,ip,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,...,thrcnt_week,thrcnt_day,p6,p9,p5m,p5w,p5d,p8m,p8w,p8d
0,Nhq,DPM,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,3,...,4160,675,1,0,2,1,1,1,1,1
1,XZt,FIN,0,Exploit,192.SL.UK.94,PRIV-192,Private network,1,A,5,...,9,2,4,12,3,2,2,2,1,1
2,bBz,CHP,0,Attack,YT.LB.38.21,INTERNET,Internet,7,A,4,...,3788,628,1,0,2,2,1,2,2,1
3,ZNr,HPS,0,Attack,JX.NY.13.20,INTERNET,Internet,7,A,4,...,565,96,0,0,2,2,2,2,2,2
4,poV,OSC,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,4,...,2790,632,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39422,cWL,LSB,0,Attack,MC.ER.197.27,INTERNET,Internet,7,A,5,...,2742,579,1,0,2,1,1,2,1,1
39423,kaP,ESS,0,Control and Maintain,10.XU.AD.142,PRIV-10,Private network,1,A,3,...,7,4,0,0,3,2,1,3,2,1
39424,hLv,TDM,0,Attack,YT.LB.38.21,INTERNET,Internet,7,A,4,...,2631,253,1,0,1,1,1,1,1,1
39425,sWM,IUO,0,Exploit,NZ.XH.108.109,INTERNET,Internet,7,A,4,...,6,2,0,0,1,1,1,1,1,1


Read testing file and label. Combine them into one data frame

In [3]:
test_df = pd.read_csv('cybersecurity_test.csv', sep='|')
with open('cybersecurity_test_targets.txt', 'r') as f:
  test_targets = f.readlines()
# Remove newline characters from test_targets and convert to a list of integers
test_targets = [int(target.strip()) for target in test_targets]

# Create a new column 'notified' in the test_df DataFrame with the values from test_targets
test_df['notified'] = test_targets

In [4]:
# Concatenate train and test_df DataFrames
full_df = pd.concat([train, test_df], axis=0)

# Reset the index of the combined DataFrame
full_df = full_df.reset_index(drop=True)

full_df


,alert_ids,client_code,notified,categoryname,ip,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,...,thrcnt_week,thrcnt_day,p6,p9,p5m,p5w,p5d,p8m,p8w,p8d
0,Nhq,DPM,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,3,...,4160,675,1,0,2,1,1,1,1,1
1,XZt,FIN,0,Exploit,192.SL.UK.94,PRIV-192,Private network,1,A,5,...,9,2,4,12,3,2,2,2,1,1
2,bBz,CHP,0,Attack,YT.LB.38.21,INTERNET,Internet,7,A,4,...,3788,628,1,0,2,2,1,2,2,1
3,ZNr,HPS,0,Attack,JX.NY.13.20,INTERNET,Internet,7,A,4,...,565,96,0,0,2,2,2,2,2,2
4,poV,OSC,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,4,...,2790,632,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,YT.LB.133.99,INTERNET,Internet,7,A,2,...,75,4,1,0,1,1,1,1,1,1
59423,Nty,BMU,0,Exploit,MA.LA.169.242,INTERNET,Internet,7,A,3,...,39,8,1,0,2,2,1,2,2,2
59424,TmA,PCV,0,Exploit,10.NF.RC.138,PRIV-10,Private network,1,A,5,...,1,1,1,0,1,1,1,1,1,1
59425,zkv,JIT,1,Attack,KN.IR.6.26,INTERNET,Internet,7,A,3,...,15,1,1,0,3,2,1,3,2,1


# Data Pre-processing

## First Data Set

In [5]:
full_df[['ip_1', 'ip_2', 'ip_3', 'ip_4']] = full_df['ip'].str.split('.', expand=True)

# Drop the original 'ip' column
full_df = full_df.drop('ip', axis=1)

In [6]:
full_df['ip_1_ip_2'] = full_df['ip_1'] + ' ' + full_df['ip_2']

In [7]:
# Count the occurrences of each value in 'ip_1', 'ip_2', 'ip_3', and 'ip_4'
ip_1_counts = full_df['ip_1'].value_counts()
ip_2_counts = full_df['ip_2'].value_counts()

ip_1_2_counts = full_df['ip_1_ip_2'].value_counts()

print("Top 20 most frequent values in ip_1:\n", ip_1_counts.head(20))
print("\nTop 20 most frequent values in ip_2:\n", ip_2_counts.head(20))

print("\nTop 20 most frequent values in ip_1_2_counts:\n", ip_1_2_counts.head(20))

Top 20 most frequent values in ip_1:
 ip_1
YT     15437
10     14517
192     3674
172     2854
MC      2399
OQ      2037
FX       940
YA       787
SO       687
BZ       676
DJ       631
BI       570
ZU       531
IA       501
JX       492
JM       488
EE       448
100      411
QP       394
ON       332
Name: count, dtype: int64

Top 20 most frequent values in ip_2:
 ip_2
LB    14698
SL     3695
ER     3432
QJ     1838
KW     1003
JE      640
CN      640
YT      617
BW      571
RD      564
SK      548
TJ      498
TY      494
NF      464
BA      432
LZ      417
BH      414
JJ      383
TK      368
TV      342
Name: count, dtype: int64

Top 20 most frequent values in ip_1_2_counts:
 ip_1_ip_2
YT LB     14629
192 SL     3674
MC ER      2141
OQ QJ      1707
172 ER     1140
10 KW      1003
10 CN       640
YT RD       540
ZU SK       496
10 TJ       494
BZ TY       466
SO JE       461
YA YT       422
100 LZ      392
10 BH       388
10 NF       371
IA TK       357
JX BA       346
10 BW       332

In [8]:
top_20_ip_1 = ip_1_counts.head(20).index.tolist()
top_20_ip_2 = ip_2_counts.head(20).index.tolist()
top_20_ip_1_2 = ip_1_2_counts.head(20).index.tolist()

# Replace values not in the top 10 with 'other'
full_df['ip_1'] = full_df['ip_1'].apply(lambda x: x if x in top_20_ip_1 else 'other')
full_df['ip_2'] = full_df['ip_2'].apply(lambda x: x if x in top_20_ip_2 else 'other')
full_df['ip_1_ip_2'] = full_df['ip_1_ip_2'].apply(lambda x: x if x in top_20_ip_1_2 else 'other')


In [9]:
full_df = full_df.drop(['ip_3', 'ip_4'], axis=1)

In [10]:
# Calculate the sum of p5w, p5d, and p5m
full_df['p5'] = full_df['p5w'] + full_df['p5d'] + full_df['p5m']

# Calculate the sum of p8m, p8d, and p8w
full_df['p8'] = full_df['p8m'] + full_df['p8d'] + full_df['p8w']

# Drop the original columns
full_df = full_df.drop(['p5w', 'p5d', 'p5m', 'p8m', 'p8d', 'p8w'], axis=1)

# Display the updated DataFrame
full_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,thrcnt_month,thrcnt_week,thrcnt_day,p6,p9,ip_1,ip_2,ip_1_ip_2,p5,p8
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,16711,4160,675,1,0,YT,LB,YT LB,4,3
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,15,9,2,4,12,192,SL,192 SL,7,4
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,7393,3788,628,1,0,YT,LB,YT LB,5,5
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,2048,565,96,0,0,JX,other,other,6,6
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,2793,2790,632,1,0,YT,LB,YT LB,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,290,75,4,1,0,YT,LB,YT LB,3,3
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,295,39,8,1,0,other,other,other,5,6
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,7,1,1,1,0,10,NF,10 NF,3,3
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,40,15,1,1,0,other,other,other,6,6


In [11]:
# combine the time

def categorize_time(hour):
  if 6 <= hour <= 12:
    return 'Morning'
  elif 12 < hour <= 18:
    return 'Afternoon'
  elif 18 < hour <= 21:
    return 'Evening'
  else:
    return 'Night'

full_df['dayPeriods'] = full_df['start_hour'].apply(categorize_time)

In [12]:
full_df = full_df.drop(['start_hour', 'start_minute', 'start_second'], axis=1)
full_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,thrcnt_week,thrcnt_day,p6,p9,ip_1,ip_2,ip_1_ip_2,p5,p8,dayPeriods
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,4160,675,1,0,YT,LB,YT LB,4,3,Morning
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,9,2,4,12,192,SL,192 SL,7,4,Night
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,3788,628,1,0,YT,LB,YT LB,5,5,Afternoon
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,565,96,0,0,JX,other,other,6,6,Evening
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,2790,632,1,0,YT,LB,YT LB,3,3,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,75,4,1,0,YT,LB,YT LB,3,3,Evening
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,39,8,1,0,other,other,other,5,6,Afternoon
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,1,1,1,0,10,NF,10 NF,3,3,Afternoon
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,15,1,1,0,other,other,other,6,6,Morning


In [13]:
# Check if there is null value in these column，and set a new column to record it.
columns_to_check = ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'score']

for column in columns_to_check:
  new_column_name = column + '_isNull'
  full_df[new_column_name] = full_df[column].isnull().astype(int)

full_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,n2_isNull,n3_isNull,n4_isNull,n5_isNull,n6_isNull,n7_isNull,n8_isNull,n9_isNull,n10_isNull,score_isNull
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,1,1,1,1,1,1,1,1,1,1
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,1,1,1,1,1,1,1,1,1,1
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,1,1,1,1,1,1,1,1,1,1
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,1,1,1,1,1,1,1,1,1,1
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0,0,0,0,0,0,0,0,0,0
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0,0,0,0,0,0,0,0,0,0
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0,0,0,0,0,0,0,0,0,0
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Fill missing values in specified columns with -1
columns_to_fill = ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'score']
full_df[columns_to_fill] = full_df[columns_to_fill].fillna(-1)
full_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,n2_isNull,n3_isNull,n4_isNull,n5_isNull,n6_isNull,n7_isNull,n8_isNull,n9_isNull,n10_isNull,score_isNull
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,1,1,1,1,1,1,1,1,1,1
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,1,1,1,1,1,1,1,1,1,1
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,1,1,1,1,1,1,1,1,1,1
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,1,1,1,1,1,1,1,1,1,1
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0,0,0,0,0,0,0,0,0,0
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0,0,0,0,0,0,0,0,0,0
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0,0,0,0,0,0,0,0,0,0
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0,0,0,0,0,0,0,0,0,0


## Second Data Set

In [16]:
aleart = pd.read_csv('localized_alerts_data.csv',sep='|')
aleart

C:\Users\孙嘉淇\AppData\Local\Temp\ipykernel_27776\3539115333.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  aleart = pd.read_csv('localized_alerts_data.csv',sep='|')


,alert_ids,alerttype,devicetype,reportingdevice_code,devicevendor_code,srcip,dstip,srcipcategory,dstipcategory,srcport,...,srcportcategory,dstportcategory,direction,alerttime,severity,count,domain,protocol,username,signature
0,AAB,NAC: Asset Visibility,NAC,Wji,QO,10.NF.KO.199,NaN,PRIV-10,INTERNET,0.0,...,1,1,5,0,1,1,0,NaN,1,1
1,AAC,ThreatWatch Outbound,FW,GFv,JS,192.SL.XK.61,YT.LB.32.21,PRIV-192,INTERNET,63723.0,...,4,2,3,0,5,19,0,NaN,1,1
2,AAE,ThreatWatch Outbound,FW,tMU,SX,10.CN.AE.137,EB.QD.27.77,PRIV-10,INTERNET,59806.0,...,4,3,3,0,3,10,0,NaN,0,1
3,AAL,ThreatWatch Outbound,FW,xQn,SX,172.AT.TL.37,YT.LB.34.21,PRIV-172,INTERNET,63496.0,...,4,2,3,0,5,1,0,NaN,0,1
4,AAL,ThreatWatch Inbound,FW,xQn,SX,YT.LB.34.21,YT.EK.108.146,INTERNET,INTERNET,443.0,...,2,4,2,1311,5,1,0,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8690699,zzu,Suspicious Port Activity,FW,KcT,SX,192.SL.BZ.45,BZ.WI.74.19,PRIV-192,INTERNET,18755.0,...,3,2,3,222527,1,20,0,NaN,0,1
8690700,zzu,Suspicious Port Activity,FW,KcT,SX,192.SL.BZ.45,BZ.WI.74.26,PRIV-192,INTERNET,18389.0,...,3,2,3,222580,1,83,0,NaN,0,1
8690701,zzx,IDPS Alert,NIPS,sZp,BU,10.NF.RH.9,BW.LU.214.37,PRIV-10,INTERNET,29539.0,...,3,2,3,0,5,2,0,6,0,1
8690702,zzx,IDPS Alert,NIDS,WmQ,TR,10.NF.RH.9,BW.LU.214.37,PRIV-10,INTERNET,29539.0,...,3,2,3,8,5,2,0,TCP,1,1


In [17]:
# Group by 'alerttype' and 'alert_ids', then count the occurrences
alert_counts = aleart.groupby(['alerttype', 'alert_ids'])['alert_ids'].count().reset_index(name='count')
# Display the result
alert_counts

,alerttype,alert_ids,count
0,ADPS - Adware,Zih,1
1,ADPS - Dynamic DNS,ZLX,1
2,ADPS - Dynamic DNS,sgb,1
3,ADPS - High Risk Domains,Gfa,1
4,ADPS - High Risk Domains,ILc,1
...,...,...,...
72211,Wanna Cry Kill Switch Communication,xkI,1
72212,Wanna Cry Kill Switch Communication,yGV,1
72213,Wanna Cry Kill Switch Communication,yMp,1
72214,Wanna Cry Kill Switch Communication,zIg,6


In [18]:
alert_pivot = alert_counts.pivot(index='alert_ids', columns='alerttype', values='count').fillna(0)
# Merge the data to original data set
merged_df = full_df.merge(alert_pivot, on='alert_ids', how='left').fillna(0)
merged_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,Suspicious Outbound Anomaly - Company,Suspicious Outbound Anomaly - Workstation,Suspicious Port Activity,Suspicious SMB Activity,Targeted Attack,ThreatWatch Inbound,ThreatWatch Outbound,User Password Changed,WAF Alert,Wanna Cry Kill Switch Communication
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,0.0,0.0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,0.0
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,0.0,12.0,5220.0,9.0,0.0,0.0,61.0,0.0,0.0,0.0
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,0.0,0.0,0.0,0.0,0.0,0.0,346.0,0.0,0.0,0.0
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
devicetype_counts = aleart.groupby(['devicetype', 'alert_ids'])['alert_ids'].count().reset_index(name='count')
# Display the result
devicetype_counts

,devicetype,alert_ids,count
0,AAA,Bib,1
1,AAA,BsL,7
2,AAA,DDk,2
3,AAA,DLV,1
4,AAA,Ekw,1
...,...,...,...
61876,WS,zrF,2
61877,WS,zwq,5
61878,WS,zyc,7
61879,ePO,KKV,2


In [20]:
devicetype_pivot = devicetype_counts.pivot(index='alert_ids', columns='devicetype', values='count').fillna(0)
# Merge the data to original data set
merged_df = merged_df.merge(devicetype_pivot, on='alert_ids', how='left').fillna(0)
merged_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,NX,OS,RS,SGFW,Scan,Scan Surveillance,TPS,WAF,WS,ePO
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Group by 'reportingdevice_code' and sum the 'severity' values
severity_sum_by_device = aleart.groupby('reportingdevice_code')['severity'].sum().reset_index()

# Merge the severity sum with the original DataFrame based on 'reportingdevice_code'
merged = aleart.merge(severity_sum_by_device, on='reportingdevice_code', suffixes=('', '_sum_reportingdevice_code'))

# Create a new DataFrame with 'alert_ids' and the corresponding severity sum
new_df = merged[['alert_ids', 'severity_sum_reportingdevice_code']].drop_duplicates()

# Display the new DataFrame
new_df


,alert_ids,severity_sum_reportingdevice_code
0,AAB,68
1,AAC,27461
2,AAE,10285
3,AAL,2478
6,AAN,182485
...,...,...
8642659,zzt,363404
8642660,zzu,2577
8642685,zzx,182485
8642686,zzx,178936


In [22]:
# Group by 'alert_ids' and sum the 'severity_sum_reportingdevice_code' for each group
summed_severity = new_df.groupby('alert_ids')['severity_sum_reportingdevice_code'].sum()

# Display the result
summed_severity

alert_ids
AAB        68
AAC     27461
AAE     10285
AAL      2478
AAN    361421
        ...  
zzs    348296
zzt    363404
zzu      2577
zzx    361421
zzz      4208
Name: severity_sum_reportingdevice_code, Length: 59427, dtype: int64

In [23]:
# Merge summed_severity with merged_df on 'alert_ids'
merged_df = merged_df.merge(summed_severity, on='alert_ids', how='left').fillna(0)

# Now merged_df contains the 'severity_sum_reportingdevice_code' for each 'alert_ids'
merged_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,OS,RS,SGFW,Scan,Scan Surveillance,TPS,WAF,WS,ePO,severity_sum_reportingdevice_code
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10037
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18349
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,453766
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3254
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,635790
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,361421
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11220


In [24]:
devicevendor_counts = aleart.groupby(['devicevendor_code', 'alert_ids'])['alert_ids'].count().reset_index(name='count')

# Display the result
devicevendor_counts

,devicevendor_code,alert_ids,count
0,AE,ATJ,2
1,AE,AWP,6
2,AE,Bmt,3
3,AE,Btc,5
4,AE,CBi,4
...,...,...,...
63469,ZX,xxH,2
63470,ZX,yuS,11
63471,ZX,yyp,8
63472,ZX,zCS,15


In [25]:
devicevendor_pivot = devicevendor_counts.pivot(index='alert_ids', columns='devicevendor_code', values='count').fillna(0)
# Merge the data to original data set
merged_df = merged_df.merge(devicevendor_pivot, on='alert_ids', how='left').fillna(0)
merged_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,TA,TR,VN,VX,WX,YC,YL,ZD,ZT,ZX
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
protocol_counts = aleart.groupby(['protocol', 'alert_ids'])['alert_ids'].count().reset_index(name='count')

# Display the result
protocol_counts

,protocol,alert_ids,count
0,6,YSZ,16
1,6,sqH,5
2,17,YSZ,98288
3,17,sqH,65524
4,1,ArP,31
...,...,...,...
45242,udp/ip,ocj,2
45243,udp/ip,xrB,826
45244,vosaic-d,BkS,1
45245,vpn,RTS,1


In [27]:
def clean_protocol(protocol):
  protocol = str(protocol).lower()
  if 'tcp' in protocol:
    return 'tcp'
  elif 'udp' in protocol:
    return 'udp'
  elif 'ssh' in protocol:
    return 'ssh'
  elif 'http' in protocol and 'https' not in protocol:
    return 'http'
  elif 'https' in protocol:
    return 'https'
  elif 'ftp' in protocol:
    return 'ftp'
  elif 'dns' in protocol:
    return 'dns'
  elif 'imap' in protocol:
    return 'imap'
  elif 'pop' in protocol:
    return 'pop'
  elif 'smtp' in protocol:
    return 'smtp'
  elif 'ntp' in protocol:
    return 'ntp'
  elif 'vpn' in protocol:
    return 'vpn'
  elif protocol.isdigit():
    return 'numerical_protocol'
  else:
    return 'other'

protocol_counts['protocol'] = protocol_counts['protocol'].apply(clean_protocol)


protocol_counts

,protocol,alert_ids,count
0,numerical_protocol,YSZ,16
1,numerical_protocol,sqH,5
2,numerical_protocol,YSZ,98288
3,numerical_protocol,sqH,65524
4,numerical_protocol,ArP,31
...,...,...,...
45242,udp,ocj,2
45243,udp,xrB,826
45244,other,BkS,1
45245,vpn,RTS,1


In [28]:
aggregated_counts = protocol_counts.groupby(['alert_ids', 'protocol']).sum().reset_index()

In [29]:
protocol_pivot = aggregated_counts.pivot(index='alert_ids', columns='protocol', values='count').fillna(0)
merged_df = merged_df.merge(protocol_pivot, on='alert_ids', how='left').fillna(0)
merged_df


,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,imap,ntp,numerical_protocol,other,pop,smtp,ssh,tcp,udp,vpn
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,346.0,0.0,0.0
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Group by 'alert_ids' and calculate the sum, mean, max, median, and min of 'severity'
severity_stats = aleart.groupby('alert_ids')['severity'].agg(['sum', 'mean', 'max', 'median', 'min']).reset_index()

# Rename the columns for better readability
severity_stats.rename(columns={'sum': 'severity_sum', 'mean': 'severity_mean', 'max': 'severity_max', 'median': 'severity_median', 'min': 'severity_min'}, inplace=True)

# Display the resulting DataFrame
severity_stats

,alert_ids,severity_sum,severity_mean,severity_max,severity_median,severity_min
0,AAB,1,1.0,1,1.0,1
1,AAC,5,5.0,5,5.0,5
2,AAE,3,3.0,3,3.0,3
3,AAL,15,5.0,5,5.0,5
4,AAN,30,5.0,5,5.0,5
...,...,...,...,...,...,...
59422,zzs,690,5.0,5,5.0,5
59423,zzt,2,2.0,2,2.0,2
59424,zzu,25,1.0,1,1.0,1
59425,zzx,10,5.0,5,5.0,5


In [31]:
# Merge the severity statistics with the existing merged_df
merged_df = merged_df.merge(severity_stats, on='alert_ids', how='left')

# Display the updated merged_df
merged_df

,alert_ids,client_code,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,smtp,ssh,tcp,udp,vpn,severity_sum,severity_mean,severity_max,severity_median,severity_min
0,Nhq,DPM,0,Attack,INTERNET,Internet,7,A,3,65684,...,0.0,0.0,0.0,0.0,0.0,138,2.000000,2,2.0,2
1,XZt,FIN,0,Exploit,PRIV-192,Private network,1,A,5,1188030,...,0.0,0.0,0.0,0.0,0.0,5533,1.043568,5,1.0,1
2,bBz,CHP,0,Attack,INTERNET,Internet,7,A,4,43716,...,0.0,0.0,346.0,0.0,0.0,1384,4.000000,4,4.0,4
3,ZNr,HPS,0,Attack,INTERNET,Internet,7,A,4,0,...,0.0,0.0,1.0,0.0,0.0,5,5.000000,5,5.0,5
4,poV,OSC,0,Attack,INTERNET,Internet,7,A,4,2401,...,0.0,0.0,0.0,0.0,0.0,5,5.000000,5,5.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,ZRV,0,Exploit,INTERNET,Internet,7,A,2,43201,...,0.0,0.0,0.0,0.0,0.0,3,1.000000,1,1.0,1
59423,Nty,BMU,0,Exploit,INTERNET,Internet,7,A,3,0,...,0.0,0.0,1.0,0.0,0.0,3,3.000000,3,3.0,3
59424,TmA,PCV,0,Exploit,PRIV-10,Private network,1,A,5,1,...,0.0,0.0,1.0,0.0,0.0,10,5.000000,5,5.0,5
59425,zkv,JIT,1,Attack,INTERNET,Internet,7,A,3,748,...,0.0,0.0,0.0,0.0,0.0,4,4.000000,4,4.0,4


In [32]:
merged_df = merged_df.drop(columns='client_code')

In [33]:
alerttime_stats = aleart.groupby('alert_ids')['alerttime'].agg(['sum', 'mean', 'max', 'min']).reset_index()

# Rename the columns for better readability
alerttime_stats.rename(columns={'sum': 'alerttime_sum', 'mean': 'alerttime_mean', 'max': 'alerttime_max', 'min': 'alerttime_min'}, inplace=True)

# Display the resulting DataFrame
alerttime_stats

,alert_ids,alerttime_sum,alerttime_mean,alerttime_max,alerttime_min
0,AAB,0,0.000000,0,0
1,AAC,0,0.000000,0,0
2,AAE,0,0.000000,0,0
3,AAL,2630,876.666667,1319,0
4,AAN,10178,1696.333333,3100,0
...,...,...,...,...,...
59422,zzs,3699944,26811.188406,71491,0
59423,zzt,0,0.000000,0,0
59424,zzu,2990383,119615.320000,222580,0
59425,zzx,8,4.000000,8,0


In [34]:
merged_df = merged_df.merge(alerttime_stats, on='alert_ids', how='left')

# Display the updated merged_df
merged_df

,alert_ids,notified,categoryname,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,weekday,...,vpn,severity_sum,severity_mean,severity_max,severity_median,severity_min,alerttime_sum,alerttime_mean,alerttime_max,alerttime_min
0,Nhq,0,Attack,INTERNET,Internet,7,A,3,65684,Tue,...,0.0,138,2.000000,2,2.0,2,2244595,32530.362319,63323,0
1,XZt,0,Exploit,PRIV-192,Private network,1,A,5,1188030,Thu,...,0.0,5533,1.043568,5,1.0,1,3129027554,590159.855526,1188000,0
2,bBz,0,Attack,INTERNET,Internet,7,A,4,43716,Tue,...,0.0,1384,4.000000,4,4.0,4,6812661,19689.771676,43716,0
3,ZNr,0,Attack,INTERNET,Internet,7,A,4,0,Tue,...,0.0,5,5.000000,5,5.0,5,0,0.000000,0,0
4,poV,0,Attack,INTERNET,Internet,7,A,4,2401,Sat,...,0.0,5,5.000000,5,5.0,5,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,rub,0,Exploit,INTERNET,Internet,7,A,2,43201,Sun,...,0.0,3,1.000000,1,1.0,1,51602,17200.666667,30601,0
59423,Nty,0,Exploit,INTERNET,Internet,7,A,3,0,Thu,...,0.0,3,3.000000,3,3.0,3,0,0.000000,0,0
59424,TmA,0,Exploit,PRIV-10,Private network,1,A,5,1,Sat,...,0.0,10,5.000000,5,5.0,5,1,0.500000,1,0
59425,zkv,1,Attack,INTERNET,Internet,7,A,3,748,Mon,...,0.0,4,4.000000,4,4.0,4,0,0.000000,0,0


## Export the full data set

In [35]:
merged_df.to_csv('merged_data_full.csv', index=False)

## Split the training set and testing set

In [37]:

df = pd.read_csv('merged_data_full.csv')


test_alert_ids = set(test_df['alert_ids'])


testset = df[df['alert_ids'].isin(test_alert_ids)]
trainingset = df[~df['alert_ids'].isin(test_alert_ids)]


print(f"Test set size: {len(testset)}")
print(f"Training set size: {len(trainingset)}")

Test set size: 20000
Training set size: 39427


In [38]:
testset.to_csv('test_modified.csv', index=False)
trainingset.to_csv('train_modified.csv', index=False)

## One Hot Encoding

In [39]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from scipy.stats import entropy
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
test_df = pd.read_csv('cybersecurity_test.csv', sep='|')
df = pd.read_csv('merged_data_full.csv')

test_alert_ids = set(test_df['alert_ids'])

testset = df[df['alert_ids'].isin(test_alert_ids)]
trainingset = df[~df['alert_ids'].isin(test_alert_ids)]

print(f"Test set size: {len(testset)}")
print(f"Training set size: {len(trainingset)}")

Test set size: 20000
Training set size: 39427


In [40]:
df_X = df.drop("notified", axis=1)
df_y = df["notified"]
categorical_features = ["categoryname", "ipcategory_name", "ipcategory_scope", "parent_category",
                       "grandparent_category", "weekday", "isiptrusted", "enforcementscore",
                       "dstipcategory_dominate", "srcipcategory_dominate", "dstportcategory_dominate",
                       "srcportcategory_dominate", "ip_1", "ip_2", "ip_1_ip_2", "dayPeriods",
                       "n1_isNull", "n2_isNull", "n3_isNull", "n4_isNull", "n5_isNull", "n6_isNull",
                       "n7_isNull", "n8_isNull", "n9_isNull", "n10_isNull", "score_isNull"]


X_N = df_X.drop(categorical_features, axis=1)

X_C = df_X[categorical_features]
onehotencoder = OneHotEncoder(sparse_output=False)

column_transformer = ColumnTransformer(
    transformers=[
        ('cat', onehotencoder, categorical_features)
    ],
    remainder='passthrough'  
)

X_encoded = column_transformer.fit_transform(X_C)

encoded_X_C = pd.DataFrame(X_encoded, columns=column_transformer.get_feature_names_out())
encoded_X_C["notified"] = df_y

In [41]:
merged_df = pd.concat([encoded_X_C, X_N], axis=1)
merged_df

,cat__categoryname_Attack,cat__categoryname_Attack Preparation,cat__categoryname_Compromise,cat__categoryname_Control and Maintain,cat__categoryname_Exploit,cat__categoryname_Malicious Activity,cat__categoryname_Reconnaissance,cat__categoryname_Suspicious Account Activity,cat__categoryname_Suspicious Network Activity,cat__categoryname_Suspicious Reputation,...,vpn,severity_sum,severity_mean,severity_max,severity_median,severity_min,alerttime_sum,alerttime_mean,alerttime_max,alerttime_min
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,138,2.000000,2,2.0,2,2244595,32530.362319,63323,0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5533,1.043568,5,1.0,1,3129027554,590159.855526,1188000,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1384,4.000000,4,4.0,4,6812661,19689.771676,43716,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5,5.000000,5,5.0,5,0,0.000000,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5,5.000000,5,5.0,5,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,1.000000,1,1.0,1,51602,17200.666667,30601,0
59423,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,3.000000,3,3.0,3,0,0.000000,0,0
59424,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10,5.000000,5,5.0,5,1,0.500000,1,0
59425,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4,4.000000,4,4.0,4,0,0.000000,0,0


In [42]:
merged_df.to_csv('merged_df_onehot.csv', index=False)

In [43]:
test_alert_ids = set(test_df['alert_ids'])

test = merged_df[merged_df['alert_ids'].isin(test_alert_ids)]
train = merged_df[~merged_df['alert_ids'].isin(test_alert_ids)]
train

,cat__categoryname_Attack,cat__categoryname_Attack Preparation,cat__categoryname_Compromise,cat__categoryname_Control and Maintain,cat__categoryname_Exploit,cat__categoryname_Malicious Activity,cat__categoryname_Reconnaissance,cat__categoryname_Suspicious Account Activity,cat__categoryname_Suspicious Network Activity,cat__categoryname_Suspicious Reputation,...,vpn,severity_sum,severity_mean,severity_max,severity_median,severity_min,alerttime_sum,alerttime_mean,alerttime_max,alerttime_min
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,138,2.000000,2,2.0,2,2244595,32530.362319,63323,0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5533,1.043568,5,1.0,1,3129027554,590159.855526,1188000,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1384,4.000000,4,4.0,4,6812661,19689.771676,43716,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5,5.000000,5,5.0,5,0,0.000000,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5,5.000000,5,5.0,5,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39422,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25,5.000000,5,5.0,5,6558,1311.600000,3146,0
39423,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6,3.000000,3,3.0,3,14778,7389.000000,14778,0
39424,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10,5.000000,5,5.0,5,16648,8324.000000,16648,0
39425,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5,5.000000,5,5.0,5,0,0.000000,0,0


In [44]:
test

,cat__categoryname_Attack,cat__categoryname_Attack Preparation,cat__categoryname_Compromise,cat__categoryname_Control and Maintain,cat__categoryname_Exploit,cat__categoryname_Malicious Activity,cat__categoryname_Reconnaissance,cat__categoryname_Suspicious Account Activity,cat__categoryname_Suspicious Network Activity,cat__categoryname_Suspicious Reputation,...,vpn,severity_sum,severity_mean,severity_max,severity_median,severity_min,alerttime_sum,alerttime_mean,alerttime_max,alerttime_min
39427,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,3.0,3,3.0,3,0,0.000000,0,0
39428,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5,5.0,5,5.0,5,0,0.000000,0,0
39429,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2,2.0,2,2.0,2,0,0.000000,0,0
39430,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2,2.0,2,2.0,2,0,0.000000,0,0
39431,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,42,3.0,3,3.0,3,1847076,131934.000000,248848,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59422,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,1.0,1,1.0,1,51602,17200.666667,30601,0
59423,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,3.0,3,3.0,3,0,0.000000,0,0
59424,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10,5.0,5,5.0,5,1,0.500000,1,0
59425,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4,4.0,4,4.0,4,0,0.000000,0,0


In [45]:
train.to_csv('train_onehot.csv', index=False)
test.to_csv('test_onehot.csv', index=False)